In [14]:
import websocket, json, pprint, talib, numpy
from binance.client import Client
from binance.enums import *
import config
import numpy as np
import pandas as pd

In [3]:
SOCKET = "wss://stream.binance.com:9443/ws/ethusdt@kline_1m"

RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
TRADE_SYMBOL = 'ETHUSD'
TRADE_QUANTITY = 0.05

closes = []
in_position = False

client = Client(config.KEY, config.SECRET, tld='us')

def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print(order)
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

    return True

    
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global closes, in_position
    
    print('received message')
    json_message = json.loads(message)
    pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        print("candle closed at {}".format(close))
        closes.append(float(close))
        print("closes")
        print(closes)

        if len(closes) > RSI_PERIOD:
            np_closes = numpy.array(closes)
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            print("all rsis calculated so far")
            print(rsi)
            last_rsi = rsi[-1]
            print("the current rsi is {}".format(last_rsi))

            if last_rsi > RSI_OVERBOUGHT:
                if in_position:
                    print("Overbought! Sell! Sell! Sell!")
                    # put binance sell logic here
                    order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)
                    if order_succeeded:
                        in_position = False
                else:
                    print("It is overbought, but we don't own any. Nothing to do.")
            
            if last_rsi < RSI_OVERSOLD:
                if in_position:
                    print("It is oversold, but you already own it, nothing to do.")
                else:
                    print("Oversold! Buy! Buy! Buy!")
                    # put binance buy order logic here
                    order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
                    if order_succeeded:
                        in_position = True

                
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

opened connection
received message
{'E': 1651470498143,
 'e': 'kline',
 'k': {'B': '0',
       'L': 813966167,
       'Q': '13970.53644600',
       'T': 1651470539999,
       'V': '4.88480000',
       'c': '2860.13000000',
       'f': 813966109,
       'h': '2860.14000000',
       'i': '1m',
       'l': '2859.70000000',
       'n': 59,
       'o': '2860.00000000',
       'q': '126119.96551500',
       's': 'ETHUSDT',
       't': 1651470480000,
       'v': '44.09830000',
       'x': False},
 's': 'ETHUSDT'}


False

[
[
1499040000000, # Open time
"0.01634790", # Open
"0.80000000", # High
"0.01575800", # Low
"0.01577100", # Close
"148976.11427815", # Volume
1499644799999, # Close time
"2434.19055334", # Quote asset volume
308, # Number of trades
"1756.87402397", # Taker buy base asset volume
"28.46694368", # Taker buy quote asset volume
"17928899.62484339" # Can be ignored
]
]

In [15]:
endTime = 1633060800000
interval_15 = 675000000
interval_1 = 45000000

i = 0
candles = []

client = Client(config.KEY, config.SECRET, tld='us')

while(len(candles) < 15000):
    currCandles = client.get_klines(symbol='ETHUSDT', 
                                interval=Client.KLINE_INTERVAL_15MINUTE, 
                                startTime = (endTime - interval_1), 
                                endTime = (endTime),
                                limit = 750)

    
    closes = []
    volumes = []
    
    print(currCandles)
    
    print(len(currCandles))
    
    for i in range(len(currCandles)):
        candle = currCandles[i]
        close = float(candle[4])
        volume = float(candle[5])

        if(len(closes) == 15):
            obv = talib.OBV(np.array(closes), np.array(volumes))
            rsi = talib.RSI(np.array(closes), 14)
            candle.append(rsi[-1])
            candle.append(obv[-1])
            candles.append(candle)
            closes = closes[:-1]
            volumes = volumes[:-1]

        closes.insert(0, close)
        volumes.insert(0, volume)
        
        i+=1 
        #print(i)
    print("YES", len(candles))
    endTime += interval_1
print('\n\n\nFINAL\n\n', candles)

[[1633015800000, '3006.66000000', '3009.82000000', '2985.13000000', '2991.79000000', '31.16178000', 1633016699999, '93546.44593640', 87, '19.18492000', '57661.55290770', '0'], [1633016700000, '2993.84000000', '2993.84000000', '2972.72000000', '2980.92000000', '9.31945000', 1633017599999, '27776.73033640', 52, '3.48941000', '10394.17483420', '0'], [1633017600000, '2987.83000000', '2996.63000000', '2967.80000000', '2967.81000000', '40.15359000', 1633018499999, '119619.64316530', 95, '24.47534000', '72899.24580080', '0'], [1633018500000, '2972.16000000', '2977.90000000', '2969.60000000', '2973.83000000', '13.23936000', 1633019399999, '39361.02888570', 47, '3.14990000', '9367.34450350', '0'], [1633019400000, '2967.95000000', '2977.75000000', '2964.97000000', '2970.74000000', '15.71182000', 1633020299999, '46695.69136070', 51, '9.66595000', '28735.43862300', '0'], [1633020300000, '2973.77000000', '2982.37000000', '2971.12000000', '2977.61000000', '23.65792000', 1633021199999, '70405.2880879

[[1633195800000, '3425.61000000', '3427.02000000', '3419.11000000', '3419.34000000', '13.36288000', 1633196699999, '45736.27276550', 65, '8.42526000', '28842.16444800', '0'], [1633196700000, '3424.20000000', '3438.31000000', '3419.79000000', '3430.03000000', '4.69061000', 1633197599999, '16095.06578410', 43, '2.50862000', '8613.39288320', '0'], [1633197600000, '3431.60000000', '3444.00000000', '3421.64000000', '3423.37000000', '45.56512000', 1633198499999, '155999.71059090', 74, '36.10336000', '123581.95501240', '0'], [1633198500000, '3424.58000000', '3428.32000000', '3411.77000000', '3418.46000000', '15.80080000', 1633199399999, '54045.27470870', 70, '7.09955000', '24280.97858120', '0'], [1633199400000, '3422.01000000', '3430.88000000', '3421.87000000', '3430.60000000', '7.61370000', 1633200299999, '26093.71348780', 30, '2.77371000', '9512.47079560', '0'], [1633200300000, '3428.53000000', '3443.98000000', '3428.53000000', '3443.97000000', '24.74682000', 1633201199999, '85061.71918470'

[[1633465800000, '3508.63000000', '3520.24000000', '3492.86000000', '3519.60000000', '34.82002000', 1633466699999, '121959.92599110', 165, '24.90004000', '87184.49180940', '0'], [1633466700000, '3518.95000000', '3532.02000000', '3513.58000000', '3524.35000000', '12.90951000', 1633467599999, '45486.13974640', 140, '7.76012000', '27349.51806510', '0'], [1633467600000, '3523.88000000', '3526.70000000', '3514.58000000', '3519.70000000', '13.50423000', 1633468499999, '47528.94566480', 71, '6.07959000', '21407.89966270', '0'], [1633468500000, '3524.18000000', '3524.19000000', '3511.51000000', '3522.78000000', '18.14699000', 1633469399999, '63816.97522010', 63, '6.51562000', '22937.52288750', '0'], [1633469400000, '3521.01000000', '3533.29000000', '3518.46000000', '3533.29000000', '9.61408000', 1633470299999, '33922.14017770', 58, '9.12074000', '32183.87400590', '0'], [1633470300000, '3529.05000000', '3539.26000000', '3524.00000000', '3539.00000000', '13.94717000', 1633471199999, '49265.58222

[[1633735800000, '3559.84000000', '3567.73000000', '3549.73000000', '3553.04000000', '26.91214000', 1633736699999, '95802.25175160', 75, '5.00065000', '17802.94715860', '0'], [1633736700000, '3548.77000000', '3565.86000000', '3548.77000000', '3562.52000000', '11.42379000', 1633737599999, '40672.74424910', 43, '7.54227000', '26874.58357050', '0'], [1633737600000, '3563.06000000', '3575.65000000', '3539.47000000', '3563.81000000', '27.58472000', 1633738499999, '98253.10128610', 149, '19.24659000', '68563.69182830', '0'], [1633738500000, '3565.39000000', '3571.90000000', '3548.30000000', '3569.87000000', '27.56790000', 1633739399999, '98115.39313850', 94, '19.79041000', '70442.63266440', '0'], [1633739400000, '3562.78000000', '3584.08000000', '3562.23000000', '3577.77000000', '13.59501000', 1633740299999, '48612.87313340', 86, '4.83652000', '17288.59941410', '0'], [1633740300000, '3575.68000000', '3583.38000000', '3567.50000000', '3583.38000000', '207.31246000', 1633741199999, '740331.580

[[1633960800000, '3596.63000000', '3598.05000000', '3570.01000000', '3573.11000000', '26.53419000', 1633961699999, '94933.26310740', 154, '11.46857000', '41032.48789900', '0'], [1633961700000, '3574.11000000', '3587.23000000', '3573.72000000', '3579.20000000', '42.25553000', 1633962599999, '151233.38427780', 128, '25.69386000', '91941.54036730', '0'], [1633962600000, '3578.57000000', '3598.98000000', '3578.57000000', '3595.20000000', '15.70085000', 1633963499999, '56379.30117690', 84, '11.46105000', '41158.35424180', '0'], [1633963500000, '3595.20000000', '3601.76000000', '3591.56000000', '3596.61000000', '10.43517000', 1633964399999, '37550.40741970', 58, '4.71026000', '16948.32123560', '0'], [1633964400000, '3597.88000000', '3602.57000000', '3590.17000000', '3593.79000000', '36.04426000', 1633965299999, '129651.35281780', 105, '17.23616000', '62041.85408290', '0'], [1633965300000, '3591.45000000', '3591.45000000', '3581.14000000', '3591.31000000', '10.55479000', 1633966199999, '37865

...

[[1650700800000, '2958.73000000', '2961.79000000', '2956.22000000', '2956.22000000', '1.35805000', 1650701699999, '4016.95607630', 9, '0.26787000', '793.23368070', '0'], [1650701700000, '2958.21000000', '2958.21000000', '2956.21000000', '2956.21000000', '0.72600000', 1650702599999, '2146.66046000', 2, '0.22600000', '668.55546000', '0'], [1650702600000, '2960.30000000', '2969.58000000', '2960.30000000', '2968.18000000', '8.86510000', 1650703499999, '26302.01470270', 51, '4.13815000', '12275.80283930', '0'], [1650703500000, '2968.18000000', '2968.44000000', '2961.56000000', '2963.78000000', '15.68473000', 1650704399999, '46549.39798280', 26, '1.97391000', '5858.37059500', '0'], [1650704400000, '2963.24000000', '2963.24000000', '2963.19000000', '2963.20000000', '2.41440000', 1650705299999, '7154.35950240', 5, '2.00000000', '5926.39477840', '0'], [1650705300000, '2958.64000000', '2963.02000000', '2958.21000000', '2960.71000000', '9.14720000', 1650706199999, '27081.55972450', 17, '4.9272700

[[1650925800000, '3011.90000000', '3012.26000000', '3007.19000000', '3009.26000000', '8.90657000', 1650926699999, '26799.32346100', 16, '0.43668000', '1314.89009390', '0'], [1650926700000, '3012.69000000', '3025.08000000', '3003.69000000', '3009.74000000', '29.47685000', 1650927599999, '88815.80305690', 107, '13.48372000', '40648.34802660', '0'], [1650927600000, '3008.50000000', '3011.16000000', '3004.03000000', '3010.28000000', '1.98301000', 1650928499999, '5960.14424710', 13, '0.63415000', '1907.10145130', '0'], [1650928500000, '3011.66000000', '3019.95000000', '3011.44000000', '3015.39000000', '5.87289000', 1650929399999, '17711.77778870', 26, '5.82984000', '17581.97670620', '0'], [1650929400000, '3014.93000000', '3017.78000000', '3010.01000000', '3010.01000000', '29.95488000', 1650930299999, '90311.38108210', 64, '9.86646000', '29758.01166620', '0'], [1650930300000, '3010.14000000', '3011.18000000', '3005.40000000', '3005.40000000', '17.67478000', 1650931199999, '53187.57202430', 5

[[1651150800000, '2913.84000000', '2916.92000000', '2901.05000000', '2905.41000000', '27.70151000', 1651151699999, '80497.18680110', 159, '11.92202000', '34665.32928200', '0'], [1651151700000, '2905.18000000', '2911.50000000', '2893.25000000', '2905.97000000', '57.40605000', 1651152599999, '166407.88571780', 175, '23.24507000', '67417.75224680', '0'], [1651152600000, '2912.53000000', '2915.71000000', '2887.10000000', '2889.38000000', '8.41066000', 1651153499999, '24344.00906640', 114, '3.49142000', '10123.92550420', '0'], [1651153500000, '2888.02000000', '2893.39000000', '2868.68000000', '2874.59000000', '19.03549000', 1651154399999, '54793.84180380', 132, '9.32679000', '26873.71021690', '0'], [1651154400000, '2867.60000000', '2870.82000000', '2854.96000000', '2864.32000000', '19.57875000', 1651155299999, '56041.39254050', 149, '11.52614000', '32984.76996910', '0'], [1651155300000, '2860.28000000', '2873.96000000', '2858.62000000', '2863.32000000', '13.45037000', 1651156199999, '38568.

[[1651375800000, '2774.63000000', '2774.63000000', '2768.70000000', '2771.31000000', '11.72758000', 1651376699999, '32503.79596490', 46, '7.91616000', '21941.18699160', '0'], [1651376700000, '2771.63000000', '2776.81000000', '2769.67000000', '2776.33000000', '10.41556000', 1651377599999, '28870.60826990', 42, '3.26274000', '9047.91325850', '0'], [1651377600000, '2777.58000000', '2781.49000000', '2770.62000000', '2773.08000000', '15.56330000', 1651378499999, '43202.30264100', 68, '5.65360000', '15709.33205210', '0'], [1651378500000, '2772.97000000', '2775.29000000', '2767.11000000', '2775.29000000', '7.97947000', 1651379399999, '22118.72352200', 35, '6.84591000', '18977.34748980', '0'], [1651379400000, '2772.60000000', '2774.36000000', '2769.19000000', '2770.07000000', '8.53281000', 1651380299999, '23640.69711780', 41, '4.69301000', '13004.25386370', '0'], [1651380300000, '2772.03000000', '2773.51000000', '2766.86000000', '2772.98000000', '14.65193000', 1651381199999, '40592.34300120', 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
print(len(candles))

15011


In [17]:
opens = []
closes = []
highs = []
lows = []
volumes = []
qavs = []
numTrades = []
tbbavs = []
tbqavs = []
rsis = []
obvs = []
labels = []



for i in range(len(candles) - 1):
    c = candles[i]
    
    opens.append(c[1])
    highs.append(c[2])
    lows.append(c[3])
    closes.append(c[4])
    volumes.append(c[5])
    qavs.append(c[7])
    numTrades.append(c[8])
    tbbavs.append(c[9])
    tbqavs.append(c[10])
    rsis.append(c[12])
    obvs.append(c[13])
    labels.append(1 if (candles[i+1][4] > c[4]) else 0)

In [19]:
df = pd.DataFrame(data={"opens": opens, "highs": highs, "lows": lows, "closes": closes, "volumes": volumes, "qavs": qavs, "numTrades": numTrades, "tbbavs": tbbavs, "tbqavs": tbqavs, "rsis": rsis,"obvs": obvs})
df.head()

,opens,highs,lows,closes,volumes,qavs,numTrades,tbbavs,tbqavs,rsis,obvs
0,3025.25000000,3029.31000000,3014.12000000,3014.12000000,12.27285000,37113.55734940,52,3.42848000,10363.62498610,37.733123,-145.43741
1,3019.88000000,3023.97000000,3004.00000000,3006.38000000,24.65498000,74193.58293400,68,5.14109000,15480.22724610,39.388903,-164.32634
2,3003.85000000,3009.39000000,2993.51000000,2993.51000000,36.35584000,109108.96896340,95,7.29497000,21922.18007900,37.234395,-148.99081
3,2993.14000000,3014.46000000,2993.14000000,3014.46000000,21.44103000,64442.35741320,52,8.56853000,25757.18837420,43.768997,-72.48138
4,3012.70000000,3012.70000000,3003.06000000,3003.06000000,5.13776000,15456.73568800,13,4.53611000,13648.48741600,37.564000,-136.99139


In [20]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled, columns = df.columns)
scaled_df['labels'] = labels
scaled_df.head()

joblib.dump(scaler, 'standard_scaler_1m.save') 

['standard_scaler_1m.save']

In [21]:
scaled_df.to_csv("./data_1m.csv", sep=',',index=False)

In [82]:
data = pd.read_csv('data.csv')
data.head()

,opens,highs,lows,closes,volumes,qavs,numTrades,tbbavs,tbqavs,rsis,obvs,labels
0,0.329456,0.330893,0.334030,0.334446,0.026131,0.017676,0.034536,0.015266,0.010333,0.771987,0.477127,1
1,0.335601,0.335586,0.338136,0.338821,0.047593,0.032336,0.032101,0.016821,0.011422,0.661111,0.463088,0
2,0.339230,0.333150,0.338136,0.333554,0.026950,0.018266,0.026788,0.011218,0.007601,0.651011,0.471111,0
3,0.333321,0.330141,0.335971,0.333393,0.022234,0.015054,0.020368,0.007720,0.005229,0.675504,0.468329,1
4,0.336231,0.332304,0.339276,0.336176,0.031663,0.021483,0.025459,0.020534,0.013935,0.671364,0.469930,1
